# Pretty output

Once you've settled on models, this is for printing them out for latex

In [1]:
library(lme4)
library(e1071) 
library(parameters) 
library(stargazer)
options(scipen = 100)

data <- read.csv("../../data/processed/60b37265a9f60881975de69e-reshare_rates.csv")
data$condition = as.factor(data$condition)
data$evidence = as.factor(data$evidence)
data$educationLevel = as.factor(data$educationLevel)
data$politicalAffiliation = as.factor(data$politicalAffiliation)
data$attendsProtests = as.factor(data$attendsProtests)

Loading required package: Matrix


Attaching package: ‘parameters’


The following objects are masked from ‘package:e1071’:

    kurtosis, skewness



Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 




# Confirmatory Analyses

In [8]:
md.affirms = lmer(sqrt(Affirms) ~ evidence * condition + (1 |user_id), data = data)
md.denies = lmer(sqrt(Denies) ~ evidence * condition + (1 |user_id), 
                 data = data)

In [10]:
stargazer(md.affirms,
          md.denies,
          align=TRUE,
          type = "text",
          order = c(4, 1, 2, 3),
          covariate.labels = c("Constant","(A) Low Evidence", "(B) Treatment", "(A) X (B)"),
          dep.var.labels   = c('Affirmation Rate ($p^a_R$)', "Denial Rate ($p^d_R$)"),
          title="Linear mixed-effects model demonstrating the effect of treatment on the affirmation and denial rate of rumours.",
          notes.label = "",
          add.lines=list(c("Participants", "107","107")))


Linear mixed-effects model demonstrating the effect of treatment on the affirmation and denial rate of rumours.
                           Dependent variable:        
                    ----------------------------------
                    Affirmation Rate (p Denial Rate (p
                            (1)              (2)      
------------------------------------------------------
Constant                 0.324***          0.185***   
                          (0.037)          (0.032)    
                                                      
(A) Low Evidence          -0.034           0.301***   
                          (0.038)          (0.044)    
                                                      
(B) Treatment              0.022            0.052     
                          (0.054)          (0.047)    
                                                      
(A) X (B)                 -0.056            0.008     
                          (0.055)          (0.062)    
       

## Accuracy Saliency



In [4]:
md.accSal = lmer(sqrt(Affirms) ~ condition * veracityEstimate + evidence + (1 |user_id), data = data)
stargazer(md.accSal,
          type = "text",
          title="Linear mixed-effects model demonstrating the effect of treatment and perceived rumour veracity on a rumour's affirmation rate.",
          order = c(5, 3, 1, 2,4),
          dep.var.labels   = c("Affirmation Rate ($p^a_R$)"),
          covariate.labels = c("Constant","Low Evidence", "(B) Treatment", "(C) Perceived Rumour Veracity", "(B) X (C)"),
          notes.label = "",
          add.lines=list(c("Participants", "107")),
          align=TRUE)


Linear mixed-effects model demonstrating the effect of treatment and perceived rumour veracity on a rumour's affirmation rate.
                              Dependent variable:
                              -------------------
                              Affirmation Rate (p
-------------------------------------------------
Constant                           0.287***      
                                    (0.053)      
                                                 
Low Evidence                       -0.070***     
                                    (0.026)      
                                                 
(B) Treatment                      -0.151**      
                                    (0.074)      
                                                 
(C) Perceived Rumour Veracity        0.011       
                                    (0.008)      
                                                 
(B) X (C)                          0.032***      
                      

# Exploratory Analyses

The outcome variable was the decision to reshare a given post. Since the outcome variable is a binary choice between the choice to reshare or not reshare (56 observations per participant), I employed Binomial Logistic Regression (BLR). The BLR incorporated the participant ID and post ID (unique ID from the Twitter API) as random-effects to account for repeated measures for varying items. Whether the post is misleading, defined as affirming the low evidence rumour or denying the high evidence rumour, the source gender (male vs. female profile image), and timestamp randomly assigned to each post are included as fixed-effects. Similar to Pennycook (implied truth), the control condition is used as the baseline and tests for a warning effect (or, potentially, a backfire effect if the warning hurts rather than helps) with a Warned dummy that indicates the post being in the warning treatment and having a warning, and tests for an implied truth effect with an Untagged dummy that indicates a post being in the warning treatment and not having a warning.

In [5]:
post.data <- read.csv("../../data/processed/60b37265a9f60881975de69e-rumour-results.csv")
post.data$reshared <- as.integer(as.logical(post.data$reshared))
post.data$code = relevel(as.factor(post.data$code), ref = "neutral") 
post.data$evidence = relevel(as.factor(post.data$evidence), ref = "high") 
post.data$misleading = relevel(as.factor(post.data$misleading), ref = "False") 

post.md <- glmer(reshared ~ posterGender + timestamp + warned + untagged * misleading + (1 | user_id) + (1 | id),
            data = post.data,
            family = binomial, 
            control=glmerControl(optimizer="bobyqa",optCtrl=list(maxfun=2e5))
           )

In [6]:
stargazer(post.md,
          type = "text",
          order = c(7,3,5,4,6,5,1,2),
          title="BLR mixed-effects model demonstrating the effect of post-attributes and the 'implied truth effect' on resharing odds.",
          dep.var.labels   = c("Reshared?"),
          covariate.labels = c("Constant",
                               "Warned",
                               "(D) Misleading", 
                               "(E) Untagged", 
                               "(D) X (E)", 
                               "Male",
                               "Timestamp"),
          notes.label = "",
          add.lines=list(c("Participants", "109"),c("Unique Posts","167")),
          align=TRUE)


BLR mixed-effects model demonstrating the effect of post-attributes and the 'implied truth effect' on resharing odds.
                    Dependent variable:
                    -------------------
                         Reshared?     
---------------------------------------
Constant                 -1.131***     
                          (0.195)      
                                       
Warned                    -0.141       
                          (0.255)      
                                       
(D) Misleading           -0.784***     
                          (0.188)      
                                       
(E) Untagged               0.134       
                          (0.215)      
                                       
(D) X (E)                 0.453*       
                          (0.249)      
                                       
Male                       0.011       
                          (0.076)      
                                       
T